In [2]:
!pip install -r napari_interactive_example_requirements.txt

  Attempting uninstall: laptrack
    Found existing installation: laptrack 0.1.7a2
    Uninstalling laptrack-0.1.7a2:
      Successfully uninstalled laptrack-0.1.7a2


# Initialize napari

In [25]:
import napari
import numpy as np
import pandas as pd

from IPython.display import display
from skimage.io import imread
from skimage.measure import regionprops_table
from laptrack import LapTrack
from laptrack.data_conversion import convert_dataframe_to_coords, convert_tree_to_dataframe

In [26]:
images = imread("interactive_example_data/demo_image.tif")
labels = np.load("interactive_example_data/labels3.npy")

In [27]:
viewer = napari.Viewer()
viewer.add_image(images, name="images")
viewer.add_labels(labels, name="labels")

<Labels layer 'labels' at 0x187fd3d60>

# Calculate properties of the segmentation

In [28]:
def calc_frame_regionprops(labels):  # noqa: E302
    dfs = []
    for frame in range(labels.shape[0]):
        df = pd.DataFrame(
            regionprops_table(labels[frame], properties=["label", "area", "centroid"])
        )
        df["frame"] = frame
        dfs.append(df)
    return pd.concat(dfs)
regionprops_df = calc_frame_regionprops(labels)
display(regionprops_df)

,label,area,centroid-0,centroid-1,frame
0,2,1149,21.095735,51.044386,0
1,3,676,13.699704,73.125740,0
2,4,900,13.463333,99.415556,0
3,5,706,14.866856,133.256374,0
4,6,542,9.046125,20.151292,0
...,...,...,...,...,...
34,36,905,65.306077,10.375691,9
35,37,110,14.627273,141.709091,9
36,38,741,33.570850,126.275304,9
37,39,158,133.164557,139.373418,9


# Tracking by LapTrack

## Creating data

In [29]:
_coords = convert_dataframe_to_coords(
    regionprops_df, ["centroid-0", "centroid-1", "label"]
)
coords = [c[:, :-1] for c in _coords]
coord_labels = [c[:, -1] for c in _coords]

In [37]:
coord_labels[0]

array([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
       15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
       28., 29., 30., 31., 32., 33., 34., 35.])

## Tracking

In [38]:
tree.edges()

OutEdgeView([((0, 0), (1, 0)), ((1, 0), (2, 0)), ((0, 1), (1, 1)), ((1, 1), (2, 1)), ((0, 2), (1, 2)), ((1, 2), (2, 2)), ((0, 3), (1, 3)), ((1, 3), (2, 3)), ((0, 4), (1, 4)), ((0, 4), (1, 30)), ((1, 4), (2, 4)), ((1, 30), (2, 28)), ((0, 5), (1, 5)), ((0, 6), (1, 6)), ((1, 6), (2, 5)), ((0, 7), (1, 7)), ((1, 7), (2, 6)), ((0, 8), (1, 8)), ((1, 8), (2, 7)), ((0, 10), (1, 9)), ((1, 9), (2, 8)), ((0, 11), (1, 10)), ((1, 10), (2, 9)), ((0, 12), (1, 11)), ((1, 11), (2, 10)), ((0, 13), (1, 12)), ((1, 12), (3, 9)), ((0, 14), (1, 13)), ((1, 13), (2, 11)), ((0, 15), (1, 14)), ((1, 14), (2, 12)), ((0, 16), (1, 15)), ((1, 15), (2, 13)), ((0, 17), (1, 16)), ((1, 16), (2, 14)), ((0, 18), (1, 17)), ((1, 17), (2, 15)), ((0, 19), (1, 18)), ((1, 18), (2, 16)), ((0, 20), (1, 19)), ((1, 19), (2, 17)), ((0, 21), (1, 20)), ((1, 20), (2, 18)), ((0, 22), (1, 21)), ((1, 21), (2, 19)), ((0, 23), (1, 22)), ((1, 22), (2, 20)), ((0, 24), (1, 23)), ((1, 23), (2, 21)), ((0, 25), (1, 24)), ((1, 24), (2, 22)), ((0, 26

In [35]:
lt = LapTrack(splitting_cost_cutoff=20**2)
tree = lt.predict(coords)
tracked_df, _, _ = convert_tree_to_dataframe(tree)

In [43]:
[n for n in list(tree.nodes()) if n[0]==0]

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 23),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 28),
 (0, 30),
 (0, 31),
 (0, 33)]

In [40]:
tracked_df.loc[(0,8)]

tree_id      8
track_id    10
Name: (0, 8), dtype: int64

## Adding the tracked data to the viewer

In [31]:
_regionprops_df = regionprops_df.set_index(["frame", "label"])
for (frame, index), row in tracked_df.iterrows():
    label = coord_labels[frame][index]
    _regionprops_df.loc[(frame, label), "track_id"] = row["track_id"] 
track_label_image = np.zeros_like(labels)
for (frame, label), row in _regionprops_df.iterrows():
    track_label_image[frame][labels[frame] == label] = row["track_id"] + 1

In [32]:
viewer.layers["labels"].visible = False
viewer.add_labels(track_label_image)

<Labels layer 'track_label_image' at 0x18899ad10>

# Manual correction

add points for the cells validated maually (emurated)

In [9]:
manual_corrected = np.load("interactive_example_data/manual_corrected.npy")
viewer.add_points(manual_corrected, name="manually_validated_tracks")

<Points layer 'manually_validated_tracks' at 0x1816d1d50>

In [10]:
manual_corrected = viewer.layers["manually_validated_tracks"].data.astype(np.int16)
# you can also redraw the labels
new_labels = viewer.layers["track_label_image"].data
# get label values at the placed points
validated_track_labels = new_labels[tuple(manual_corrected.T)]
validated_frames = manual_corrected[:, 0]

In [15]:
validated_points=np.array(list(zip(validated_frames, validated_track_labels)))
validated_points=validated_points[np.argsort(validated_points[:,0])]
validated_ind_pairs=[
    (((frame1,label1),(frame2,label2))) for ((frame1,label1),(frame2,label2)) 
        in zip(validated_points[:-1],validated_points[1:])
]
validated_ind_pairs

[((0, 33), (1, 33)),
 ((1, 33), (2, 35)),
 ((2, 35), (3, 35)),
 ((3, 35), (4, 35)),
 ((4, 35), (5, 35)),
 ((5, 35), (6, 35)),
 ((6, 35), (7, 35)),
 ((7, 35), (8, 35)),
 ((8, 35), (9, 35))]

# Second tracking preserving manually corrected data

In [16]:
new_regionprops_df = calc_frame_regionprops(new_labels).set_index(["frame", "label"]).reset_index()
new_coords_labels = convert_dataframe_to_coords(
    new_regionprops_df,
    ["centroid-0", "centroid-1", "label"],
)
coord_labels = [c[:, -1] for c in new_coords_labels]
new_coords = [c[:,:-1] for c in new_coords_labels]
new_regionprops_df.loc[[33,63,95]]

,frame,label,area,centroid-0,centroid-1
33,1,3,790,11.655696,95.501266
63,2,1,1098,29.715847,64.612933
95,3,2,287,5.421603,82.425087


In [22]:
get_frame_index_pair = lambda frame, label : (frame, list(coord_labels[frame]).index(label))
validated_edges = [ (get_frame_index_pair(frame1,label1),
                     get_frame_index_pair(frame2,label2)) 
    for  ((frame1,label1),(frame2,label2)) in validated_ind_pairs ]
validated_edges

[((0, 30), (1, 31)),
 ((1, 31), (2, 30)),
 ((2, 30), (3, 27)),
 ((3, 27), (4, 25)),
 ((4, 25), (5, 23)),
 ((5, 23), (6, 21)),
 ((6, 21), (7, 19)),
 ((7, 19), (8, 17)),
 ((8, 17), (9, 18))]

pairs of the coordinates of the validated points

In [23]:
lt = LapTrack(splitting_cost_cutoff=20**2)
new_tree = lt.predict(new_coords, connected_edges=validated_edges)
new_tracked_df, _, _ = convert_tree_to_dataframe(new_tree)

In [24]:
new_track_label_image = np.zeros_like(new_labels)
for (frame, ind), row in new_tracked_df.iterrows():
    label=new_labels[frame] == coord_labels[frame][ind]
    new_track_label_image[frame][label] = row["tree_id"] + 2
viewer.layers["track_label_image"].visible = False
viewer.add_labels(new_track_label_image)

<Labels layer 'new_track_label_image' at 0x1816acfd0>